In [104]:
from apscheduler.schedulers.background import BlockingScheduler
from utils.event import EventEngine, Event
from utils.brokers import Broker
from gateway.binance_http import Interval
from constant.constant import EVENT_KLINE, EVENT_POS, EVENT_OPEN_ORDERS
import pandas as pd
from strategies.boll_macd_strategy import BollMacdStrategy
import pandas_ta as ta
import numpy as np
import logging
from pandas_ta.utils import get_offset, verify_series
from decimal import Decimal
from datetime import datetime, timedelta
from transitions import Machine

In [105]:
class TradeFsm(object):

    # Define some states. Most of the time, narcoleptic superheroes are just like
    # everyone else. Except for...

        
    def __init__(self, name):
        self.name = name
        self.states = [
        { 'name': 'start', 'on_enter': ['start']},
        { 'name': 'long', 'on_enter': ['long']},
        { 'name': 'short', 'on_enter': ['short']},
        { 'name': 'monitorLong', 'on_enter': ['monitorLong']},
        { 'name': 'monitorShort', 'on_enter': ['monitorShort']},
        { 'name': 'exitLong', 'on_enter': ['exitLong']},
        { 'name': 'exitShort', 'on_enter': ['exitShort']},
        { 'name': 'stop', 'on_enter': ['stop']},
        ]
        self.transitions = [
            { 'trigger': 'longCondition', 'source': 'start', 'dest': 'long' },
            { 'trigger': 'shortCondition', 'source': 'start', 'dest': 'short' },
            { 'trigger': 'haveBuyLong', 'source': 'long', 'dest': 'monitorLong' },
            { 'trigger': 'haveBuyShort', 'source': 'short', 'dest': 'monitorShort' },
            { 'trigger': 'exitlongConditon', 'source': 'monitorLong', 'dest': 'exitLong' },
            { 'trigger': 'exitshortConditon', 'source': 'monitorShort', 'dest': 'exitShort' },
            { 'trigger': 'longWin', 'source': 'exitLong', 'dest': 'start' },
            { 'trigger': 'shortWin', 'source': 'exitShort', 'dest': 'start' },
            { 'trigger': 'longLose', 'source': 'exitLong', 'dest': 'stop' },
            { 'trigger': 'shortLose', 'source': 'exitShort', 'dest': 'stop' },
        ]

        self.machine = Machine(model = self, send_event=True, states=self.states, transitions=self.transitions, ignore_invalid_triggers=True, initial='start')

    def start(self, event):
        print("in start")
        
    def long(self, event):
        print("in long")

    def short(self, event):
        print("in short")

    def monitorLong(self, event):
        print("in monitorLong")

    def monitorShort(self, event):
        print("in monitorShort")

    def exitLong(self, event):
        print("in exitLong")

    def exitShort(self, event):
        print("in exitShort")
        
    def stop(self, event):
        print("in stop")

In [106]:
mysfp = TradeFsm("trade")
mysfp.state
mysfp.shortCondition()
mysfp.haveBuyShort()
mysfp.exitshortConditon()
mysfp.shortLose()

mysfp.machine.set_state("start")
mysfp.state
mysfp.longCondition()
mysfp.haveBuyLong()
mysfp.exitlongConditon()
mysfp.longWin()

in short
in monitorShort
in exitShort
in stop
in long
in monitorLong
in exitLong
in start


True

In [107]:
class Matter(object):
    def long(self, event):
        print("start long")
        price = event.kwargs.get('price', 0)
        num = event.kwargs.get('num', 0)
        print("price: ", price)
        print("num: ", num)
    def short(self, event): 
        print("start short")
    def monitorLong(self, event): 
        print("start monitorLong")
    
lump = Matter()
states = ['start',
         { 'name': 'long', 'on_enter': ['long']},
         { 'name': 'short', 'on_enter': ['short']},
         { 'name': 'monitorLong', 'on_enter': ['monitorLong']},
           'monitorShort',
           'exitLong',
           'exitShort',
         ]

transitions = [
    { 'trigger': 'longCondition', 'source': 'start', 'dest': 'long' },
    { 'trigger': 'shortCondition', 'source': 'start', 'dest': 'short' },
    { 'trigger': 'haveBuyLong', 'source': 'long', 'dest': 'monitorLong' },
    { 'trigger': 'haveBuyShort', 'source': 'short', 'dest': 'monitorShort' },
    { 'trigger': 'exitlongConditon', 'source': 'monitorLong', 'dest': 'exitlong' },
    { 'trigger': 'exitshortConditon', 'source': 'monitorShort', 'dest': 'exitshort' },
    { 'trigger': 'selllong', 'source': 'exitlong', 'dest': 'start' },
    { 'trigger': 'sellshort', 'source': 'exitshort', 'dest': 'start' },
]

machine = Machine(model = lump, send_event=True, states=states, transitions=transitions, ignore_invalid_triggers=True, initial='start')

#machine.set_state('asleep')
print(lump.state)
lump.shortCondition()
print(lump.state)
# lump.haveBuyLong()
# print(lump.state)
# lump.longCondition()

start
start short
short


In [108]:
format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.INFO, format=format, filename='myquant.txt')
logger = logging.getLogger('Myquant')
logging.getLogger("apscheduler").setLevel(logging.WARNING)  # 设置apscheduler.


In [109]:
def get_kline_data(symbol: str, interval: Interval):
    data = broker.binance_http.get_kline(symbol=symbol, interval=interval, limit=100)

    df = pd.DataFrame(data, columns={"open_time": 0, "open": 1, "high": 2, "low": 3, "close": 4, "volume": 5, "close_time": 6,
                               "trade_money": 7, "trade_count": 8, "buy_volume": 9, "sell_volume": 10, "other": 11})
    df = df[["open_time", "open", "high", "low", "close", "volume", "close_time"]]
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms') + pd.Timedelta(hours=8)
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms') + pd.Timedelta(hours=8)
    return df

In [110]:
key = "t3uT9bnVRXYX6OIAH2MSynTTCvHj8z9aOq1EPC70ugEMBYzyoUSko1meWFR9QJrF"
secret = '3VwOIUYNTL6So9H7Epmody6pjBj32MOazcb9WLGb384C950C65ij0wGYk8dOjOpN'

engine = EventEngine()
broker = Broker(engine, key=key, secret=secret)

data = get_kline_data('ETHUSDT', Interval.MINUTE_1)
df = data[["open_time", "open", "high", "low", "close", "volume"]]
df


,open_time,open,high,low,close,volume
0,2022-06-01 18:51:00,1934.27,1936.84,1934.14,1935.91,582.998
1,2022-06-01 18:52:00,1935.92,1936.55,1935.77,1935.77,322.832
2,2022-06-01 18:53:00,1935.76,1936.28,1935.67,1935.74,250.397
3,2022-06-01 18:54:00,1935.75,1935.81,1934.83,1935.09,163.134
4,2022-06-01 18:55:00,1935.09,1935.51,1933.57,1933.82,1241.340
...,...,...,...,...,...,...
95,2022-06-01 20:26:00,1938.89,1939.84,1938.45,1939.84,412.368
96,2022-06-01 20:27:00,1939.83,1941.54,1939.63,1941.40,2186.610
97,2022-06-01 20:28:00,1941.39,1942.07,1940.43,1940.86,1706.340
98,2022-06-01 20:29:00,1940.86,1943.00,1940.17,1942.80,1708.231


In [111]:
cycle_df = pd.DataFrame()  # 初始化一个空的DataFrame，用户接收新的数据.
rule_cycle = '2T'
df.set_index('open_time', inplace=True)
cycle_df = df.resample(rule=rule_cycle).agg(
    {'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum',
    })
cycle_df

websocket open..


,open,high,low,close,volume
open_time,,,,,
2022-06-01 18:50:00,1934.27,1936.84,1934.14,1935.91,582.998
2022-06-01 18:52:00,1935.92,1936.55,1935.67,1935.74,322.832250.397
2022-06-01 18:54:00,1935.75,1935.81,1933.57,1933.82,163.1341241.340
2022-06-01 18:56:00,1933.82,1936.77,1933.81,1936.72,595.777402.627
2022-06-01 18:58:00,1936.73,1936.73,1935.11,1935.12,370.850114.124
...,...,...,...,...,...
2022-06-01 20:22:00,1932.37,1937.26,1931.37,1936.29,715.2901299.384
2022-06-01 20:24:00,1936.27,1939.12,1936.27,1938.89,1047.331609.657
2022-06-01 20:26:00,1938.89,1941.54,1938.45,1941.40,412.3682186.610


In [112]:
def calHa(df):
    dataframe = df.copy()
    dataframe["open"]=df['open'].astype('float')
    dataframe["high"]=df['high'].astype('float')
    dataframe["low"]=df['low'].astype('float')
    dataframe["close"]=df['close'].astype('float')

    ha = ta.ha(dataframe.open, dataframe.high, dataframe.low, dataframe.close)
    ha["volume"] = df['volume']
    return ha

In [113]:
#计算1分钟的Ha
ha1min = calHa(df)

#计算2分钟的Ha
ha2min = calHa(cycle_df)
print(ha1min)
print(ha2min)

                         HA_open      HA_high       HA_low   HA_close    volume
open_time                                                                      
2022-06-01 18:51:00  1935.090000  1936.840000  1934.140000  1935.2900   582.998
2022-06-01 18:52:00  1935.190000  1936.550000  1935.190000  1936.0025   322.832
2022-06-01 18:53:00  1935.596250  1936.280000  1935.596250  1935.8625   250.397
2022-06-01 18:54:00  1935.729375  1935.810000  1934.830000  1935.3700   163.134
2022-06-01 18:55:00  1935.549687  1935.549687  1933.570000  1934.4975  1241.340
...                          ...          ...          ...        ...       ...
2022-06-01 20:26:00  1937.186073  1939.840000  1937.186073  1939.2550   412.368
2022-06-01 20:27:00  1938.220536  1941.540000  1938.220536  1940.6000  2186.610
2022-06-01 20:28:00  1939.410268  1942.070000  1939.410268  1941.1875  1706.340
2022-06-01 20:29:00  1940.298884  1943.000000  1940.170000  1941.7075  1708.231
2022-06-01 20:30:00  1941.003192  1944.0

In [114]:
#    0                       2          4          6
#[56, 58]                [58, 0]      [0, 2]      [2, 4]

utcTime = datetime.now() 
now = utcTime + timedelta(hours=7)              # UTC只是比北京时间提前了8个小时
 
 
print("now time: ", now)
minute1 = 0
minute2 = 0
if ((now.minute % 2) == 0):
    if (now.minute == 0):
        minute1 = 56
        minute2 = 58
    elif (now.minute == 2):
        minute1 = 58
        minute2 = 0
    else:
        minute1 = now.minute - 4
        minute2 = now.minute - 2
print("minute1: ", minute1)
print("minute2: ", minute2)


now time:  2022-06-01 20:30:58.888109
minute1:  26
minute2:  28


In [115]:
def findLastValue(ha, minute):
    haIndex=ha.index
    lens = len(haIndex)-1
    for index in range(lens, -1, -1):
        if haIndex[index].minute==minute:
            print('index:' + str(index))
            print('value:' + str(haIndex[index]))
            return index

In [116]:
#判断入场信号 
index1 = findLastValue(ha2min, minute1)
print(index1)
index2 = findLastValue(ha2min, minute2)
print(index2)
print(ha2min.iloc[index1])
print("===================================")
print(ha2min.iloc[index2])
ha2_index1 = ha2min.iloc[index1]
ha2_index2 = ha2min.iloc[index2]
condtion1 = ha2_index1.HA_close - ha2_index1.HA_open
condtion2 = ha2_index2.HA_close - ha2_index2.HA_open
print("condtion1: ", condtion1)
print("condtion2: ", condtion2)
long_condition = 0
short_condition = 0
if (condtion1 < 0) and (condtion2 > 0):
    print("long condition")
    long_condition = 1
elif (condtion1 > 0) and (condtion2 < 0):
    print("short condition")
    short_condition = 1
else:
    print("no trade singal")

print("===================================")
#止损止盈条件
lastBarIndex = findLastValue(ha1min, now.minute)
ha1_index = ha1min.iloc[lastBarIndex-1]
longExitCondition = 0
shortExitCondition = 0
exitCondition = ha1_index.HA_close - ha1_index.HA_open
if ((short_condition == 1) and (exitCondition > 0)):
    print("exit  short condition")
    shortExitCondition = 1
    short_condition = 0
elif ((long_condition == 1) and (exitCondition < 0)):
    print("exit long condition")
    longExitCondition = 1
    long_condition = 0
else:
    print("no exit singal")

index:48
value:2022-06-01 20:26:00
48
index:49
value:2022-06-01 20:28:00
49
HA_open             1935.69
HA_high             1941.54
HA_low              1935.69
HA_close            1940.07
volume      412.3682186.610
Name: 2022-06-01 20:26:00, dtype: object
HA_open              1937.88
HA_high                 1943
HA_low               1937.88
HA_close             1941.84
volume      1706.3401708.231
Name: 2022-06-01 20:28:00, dtype: object
condtion1:  4.37715533609844
condtion2:  3.9585776680492017
no trade singal
index:99
value:2022-06-01 20:30:00
no exit singal


In [117]:
index1 = findLastValue(ha, minute1)
print(index1)
index2 = findLastValue(ha, minute2)
print(index2)
print(ha.iloc[index1])
print("===================================")
print(ha.iloc[index2])

NameError: name 'ha' is not defined

In [118]:
#做空做多信号
ha1 = ha.iloc[index1]
ha2 = ha.iloc[index2]
condtion1 = ha1.HA_close - ha1.HA_open
condtion2 = ha2.HA_close - ha2.HA_open
print("condtion1: ", condtion1)
print("condtion2: ", condtion2)
longCondition = 0
shortConditon = 0
if (condtion1 < 0) and (condtion2 > 0):
    print("long condition")
    longCondition = 1
elif (condtion1 > 0) and (condtion2 < 0):
    print("short condition")
    shortConditon = 1
else:
    print("no trade singal")


NameError: name 'ha' is not defined

In [ ]:
sma50 = ta.sma(df.Close, length = 50)
print(sma50)

In [ ]:
sma50.plot(figsize=(16, 3), color=["black"], title="SMA_50", grid=True)

In [ ]:
mfi = ta.mfi(df.High, df.Low, df.Close, df.Volume)
df["mfi"] = mfi

In [ ]:
mfi.plot(figsize=(16, 3), color=["red"], title="mfi", grid=True)

In [ ]:
mfiSlope = ta.slope(df.mfi, as_angle=True)
mfiSlope.plot(figsize=(16, 3), color=["blue"], title="mfiSlope", grid=True)

In [ ]:
df